# Structure Data Example: Automobile dataset

https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

In [86]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [87]:
# We're using pandas to read the CSV file. This is easy for small datasets, but for large and complex datasets,
# tensorflow parsing and processing functions are more powerful
import pandas as pd
import numpy as np

# TensorFlow
import tensorflow as tf
print('please make sure that version >= 1.2:')
print(tf.__version__)
print('@monteirom: I made changes so it also works with 1.1.0 that is the current pip install version')
print('@monteirom: The lines that were changed have @1.2 as comment')

# Layers that will define the features
#
# real_value_column: real values, float32
# sparse_column_with_hash_bucket: Use this when your sparse features are in string or integer format, 
#                                 but you don't have a vocab file that maps each value to an integer ID. 
#                                 output_id = Hash(input_feature_string) % bucket_size
# sparse_column_with_keys: Look up logic is as follows: 
#                          lookup_id = index_of_feature_in_keys if feature in keys else default_value.
#                          You should use this when you know the vocab file for the feature
# one_hot_column: Creates an _OneHotColumn for a one-hot or multi-hot repr in a DNN.
#                 The input can be a _SparseColumn which is created by `sparse_column_with_*`
#                 or crossed_column functions
from tensorflow.contrib.layers import real_valued_column, sparse_column_with_keys, sparse_column_with_hash_bucket
from tensorflow.contrib.layers import one_hot_column

please make sure that version >= 1.2:
1.2.0-rc1
@monteirom: I made changes so it also works with 1.1.0 that is the current pip install version
@monteirom: The lines that were changed have @1.2 as comment


# Please Download

**https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
And move it to data/**

**So: data/imports-85.data is expected to exist!**

# Preparing the data

In [88]:
# The CSV file does not have a header, so we have to fill in column names.
names = [
    'symboling', 
    'normalized-losses', 
    'make', 
    'fuel-type', 
    'aspiration',
    'num-of-doors',
    'body-style',
    'drive-wheels',
    'engine-location',
    'wheel-base',
    'length',
    'width',
    'height',
    'curb-weight',
    'engine-type',
    'num-of-cylinders',
    'engine-size',
    'fuel-system',
    'bore',
    'stroke',
    'compression-ratio',
    'horsepower',
    'peak-rpm',
    'city-mpg',
    'highway-mpg',
    'price',
]

# We also have to specify dtypes.
dtypes = {
    'symboling': np.int32, 
    'normalized-losses': np.float32, 
    'make': str, 
    'fuel-type': str, 
    'aspiration': str,
    'num-of-doors': str,
    'body-style': str,
    'drive-wheels': str,
    'engine-location': str,
    'wheel-base': np.float32,
    'length': np.float32,
    'width': np.float32,
    'height': np.float32,
    'curb-weight': np.float32,
    'engine-type': str,
    'num-of-cylinders': str,
    'engine-size': np.float32,
    'fuel-system': str,
    'bore': np.float32,
    'stroke': np.float32,
    'compression-ratio': np.float32,
    'horsepower': np.float32,
    'peak-rpm': np.float32,
    'city-mpg': np.float32,
    'highway-mpg': np.float32,
    'price': np.float32,    
}

In [89]:
# Read the file.
df = pd.read_csv('data/imports-85.data', names=names, dtype=dtypes, na_values='?')

In [90]:
# Some rows don't have price data, we can't use those.
df = df.dropna(axis='rows', how='any', subset=['price'])

## Dealing with NaN

There are many approaches possibles for NaN values in the data, here we just changing it to " " or 0 depending of the data type. This is the simplest way, but for sure is not the best in most cases, so in practice you should try some other ways to use the NaN data. Some approaches are:

* use the mean of the row
* use the mean of the column
* if/else substituion (e.g if a lot of NaN do this, else do this other thing)
* ...
* google others


In [91]:
# Fill missing values in continuous columns with zeros instead of NaN.
float_columns = [k for k,v in dtypes.items() if v == np.float32]
df[float_columns] = df[float_columns].fillna(value=0., axis='columns')

# Fill missing values in continuous columns with '' instead of NaN (NaN mixed with strings is very bad for us).
string_columns = [k for k,v in dtypes.items() if v == str]
df[string_columns] = df[string_columns].fillna(value='', axis='columns')

## Standardize features

In [92]:
# We have too many variables let's just use some of them
df = df[['num-of-doors','num-of-cylinders', 'horsepower', 'make', 'price', 'length', 'height', 'width']]

In [93]:
# Since we're possibly dealing with parameters of different units and scales. We'll need to rescale our data.
# There are two main ways to do it: 
# * Normalization, which scales all numeric variables in the range [0,1].
#   Example:
# * Standardization, it will then transform it to have zero mean and unit variance.
#   Example: 
# Which is better? It deppends of your data and your features.
# But one disadvantage of normalization over standardization is that it loses 
# some information in the data. Since normalization loses more info it can make harder
# for gradient descent to converse, so we'll use standardization.
# In practice: please analyse your data and see what gives you better results.

def std(x):
    return (x - x.mean()) / x.std()

before = df.length[0]
df.length = std(df.length)
df.width = std(df.width)
df.height = std(df.height)
df.horsepower = std(df.horsepower)

after = df.length[0]
print('before:', before, 'after:', after)

before: 168.8 after: -0.438314


## Separating training data from testing data

In [94]:
TRAINING_DATA_SIZE = 160
TEST_DATA_SIZE = 10

LABEL = 'price'

# Split the data into a training set, eval set and test set
training_data = df[:TRAINING_DATA_SIZE]
eval_data = df[TRAINING_DATA_SIZE: TRAINING_DATA_SIZE + TEST_DATA_SIZE]
test_data = df[TRAINING_DATA_SIZE + TEST_DATA_SIZE:]

# Separate input features from labels
training_label = training_data.pop(LABEL)
eval_label = eval_data.pop(LABEL)
test_label = test_data.pop(LABEL)

# Using Tensorflow

## Defining input function

In [95]:
BATCH_SIZE = 64

# Make input function for training: 
#   num_epochs=None -> will cycle through input data forever
#   shuffle=True -> randomize order of input data
training_input_fn = tf.estimator.inputs.pandas_input_fn(x=training_data,
                                                        y=training_label,
                                                        batch_size=BATCH_SIZE,
                                                        shuffle=True,
                                                        num_epochs=None)

# Make input function for evaluation:
# shuffle=False -> do not randomize input data
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x=eval_data,
                                                    y=eval_label,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False)

# Make input function for testing:
# shuffle=False -> do not randomize input data
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x=test_data,
                                                    y=test_label,
                                                    batch_size=1,
                                                    shuffle=False)

## Defining a Linear Estimator

In [100]:
# Describe how the model should interpret the inputs. The names of the feature columns have to match the names
# of the series in the dataframe.

# @1.2.0 tf.feature_column.numeric_column -> tf.contrib.layers.real_valued_column
horsepower = real_valued_column('horsepower')
width = real_valued_column('width')
height = real_valued_column('height')
length = real_valued_column('length')

# @1.2.0 tf.feature_column.categorical_column_with_hash_bucket -> tf.contrib.layers.sparse_column_with_hash_bucket
make = sparse_column_with_hash_bucket('make', 50)

# @1.2.0 tf.feature_column.categorical_column_with_vocabulary_list -> tf.contrib.layers.sparse_column_with_keys
fuel_type = sparse_column_with_keys('fuel-type', keys=['diesel', 'gas'])
num_of_doors = sparse_column_with_keys('num-of-doors', keys=['two', 'four'])
num_of_cylinders = sparse_column_with_keys('num-of-cylinders', ['eight', 'five', 'four', 'six', 'three', 'twelve', 'two'])

linear_features = [horsepower, make, num_of_doors, num_of_cylinders, length, width, height]

In [101]:
regressor = tf.contrib.learn.LinearRegressor(feature_columns=linear_features, model_dir='tensorboard/linear_regressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_session_config': None, '_model_dir': 'tensorboard/linear_regressor/', '_master': '', '_tf_random_seed': None, '_environment': 'local', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faf67544b70>, '_save_summary_steps': 100, '_evaluation_master': '', '_is_chief': True}


## Training

In [113]:
regressor.fit(input_fn=training_input_fn, steps=10000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from tensorboard/linear_regressor/model.ckpt-10000
INFO:tensorflow:Saving checkpoints for 10001 into tensorboard/linear_regressor/model.ckpt.
INFO:tensorflow:loss = 2.3649e+08, step = 10001
INFO:tensorflow:global_step/sec: 487.666
INFO:tensorflow:loss = 2.64713e+08, step = 10101 (0.211 sec)
INFO:tensorflow:global_step/sec: 604.038
INFO:tensorflow:loss = 1.71841e+08, step = 10201 (0.163 sec)
INFO:tensorflow:global_step/sec: 753.477
INFO:tensorflow:loss = 2.28382e+08, step = 10301 (0.132 sec)
INFO:tensorflow:global_step/sec: 843.912
INFO:tensorflow:loss = 2.27272e+08, step = 10401 (0.1

INFO:tensorflow:loss = 1.88454e+08, step = 16701 (0.135 sec)
INFO:tensorflow:global_step/sec: 711.329
INFO:tensorflow:loss = 2.66716e+08, step = 16801 (0.141 sec)
INFO:tensorflow:global_step/sec: 623.028
INFO:tensorflow:loss = 2.40482e+08, step = 16901 (0.161 sec)
INFO:tensorflow:global_step/sec: 551.129
INFO:tensorflow:loss = 1.71491e+08, step = 17001 (0.181 sec)
INFO:tensorflow:global_step/sec: 665.47
INFO:tensorflow:loss = 2.43472e+08, step = 17101 (0.150 sec)
INFO:tensorflow:global_step/sec: 740.87
INFO:tensorflow:loss = 3.72998e+08, step = 17201 (0.135 sec)
INFO:tensorflow:global_step/sec: 840.003
INFO:tensorflow:loss = 1.76391e+08, step = 17301 (0.118 sec)
INFO:tensorflow:global_step/sec: 887.008
INFO:tensorflow:loss = 2.03636e+08, step = 17401 (0.113 sec)
INFO:tensorflow:global_step/sec: 886.531
INFO:tensorflow:loss = 2.03204e+08, step = 17501 (0.113 sec)
INFO:tensorflow:global_step/sec: 849.691
INFO:tensorflow:loss = 2.10635e+08, step = 17601 (0.119 sec)
INFO:tensorflow:global_

LinearRegressor(params={'joint_weights': False, 'gradient_clip_norm': None, 'feature_columns': [_RealValuedColumn(column_name='horsepower', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _SparseColumnHashed(column_name='make', is_integerized=False, bucket_size=50, lookup_config=None, combiner='sum', dtype=tf.string), _SparseColumnKeys(column_name='num-of-doors', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('two', 'four'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.string), _SparseColumnKeys(column_name='num-of-cylinders', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('eight', 'five', 'four', 'six', 'three', 'twelve', 'two'), num_oov_buckets=0, vocab_size=7, default_value=-1), combiner='sum', dtype=tf.string), _RealValuedColumn(column_name='length', dimension=1, default_value=None, dtype=tf.float32, normalizer=None),

## Evaluating

In [114]:
regressor.evaluate(input_fn=eval_input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-15-14:06:27
INFO:tensorflow:Restoring parameters from tensorboard/linear_regressor/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2017-06-15-14:06:27
INFO:tensorflow:Saving dict for global step 20000: global_step = 20000, loss = 2.01072e+08


{'global_step': 20000, 'loss': 2.0107243e+08}

## Predicting

In [115]:
preds = list(regressor.predict(input_fn=eval_input_fn))

for i in range(TEST_DATA_SIZE):
    print('prediction:', preds[i], 'real value:', test_label.iloc[i])

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from tensorboard/linear_regressor/model.ckpt-20000
prediction: 201.656 real value: 10698.0
prediction: 225.959 real value: 9988.0
prediction: 226.609 real value: 10898.0
prediction: 225.959 real value: 11248.0
prediction: 372.34 real value: 16558.0
prediction: 372.34 real value: 15998.0
prediction: 356.334 real value: 15690.0
prediction: 356.334 real value: 15750.0
prediction: 82.5563 real value: 7775.0
prediction: 125.895 real value: 7975.0


## Defining a DNN Estimator

In [105]:
# @1.2.0 tf.feature_column.indicator_column -> tf.contrib.layers.one_hot_column(tf.contrib.layers.sparse_column_with_keys(...))
dnn_features = [
    #numerical features
    length, width, height, horsepower,    
    # densify categorical features:
    one_hot_column(make),
    one_hot_column(num_of_doors)
]

In [109]:
dnnregressor = tf.contrib.learn.DNNRegressor(feature_columns=dnn_features,
                                             hidden_units=[50, 30, 10], model_dir='tensorboard/DNN_regressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_session_config': None, '_model_dir': 'tensorboard/DNN_regressor/', '_master': '', '_tf_random_seed': None, '_environment': 'local', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faf2418a630>, '_save_summary_steps': 100, '_evaluation_master': '', '_is_chief': True}


## Training

In [116]:
dnnregressor.fit(input_fn=training_input_fn, steps=10000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from tensorboard/DNN_regressor/model.ckpt-10000
INFO:tensorflow:Saving checkpoints for 10001 into tensorboard/DNN_regressor/model.ckpt.
INFO:tensorflow:loss = 3.0062e+06, step = 10001
INFO:tensorflow:global_step/sec: 410.865
INFO:tensorflow:loss = 1.80712e+06, step = 10101 (0.249 sec)
INFO:tensorflow:global_step/sec: 432.046
INFO:tensorflow:loss = 1.08074e+06, step = 10201 (0.228 sec)
INFO:tensorflow:global_step/sec: 477.219
INFO:tensorflow:loss = 2.16673e+06, step = 10301 (0.211 sec)
INFO:tensorflow:global_step/sec: 473.175
INFO:tensorflow:loss = 1.27043e+06, step = 10401 (0.211 sec

INFO:tensorflow:loss = 1.41307e+06, step = 16101 (0.215 sec)
INFO:tensorflow:global_step/sec: 445.964
INFO:tensorflow:loss = 1.54385e+06, step = 16201 (0.227 sec)
INFO:tensorflow:global_step/sec: 356.662
INFO:tensorflow:loss = 1.04547e+06, step = 16301 (0.281 sec)
INFO:tensorflow:global_step/sec: 402.729
INFO:tensorflow:loss = 1.81635e+06, step = 16401 (0.244 sec)
INFO:tensorflow:global_step/sec: 582.786
INFO:tensorflow:loss = 1.21318e+06, step = 16501 (0.170 sec)
INFO:tensorflow:global_step/sec: 604.806
INFO:tensorflow:loss = 1.47087e+06, step = 16601 (0.166 sec)
INFO:tensorflow:global_step/sec: 628.247
INFO:tensorflow:loss = 1.60473e+06, step = 16701 (0.160 sec)
INFO:tensorflow:global_step/sec: 485.227
INFO:tensorflow:loss = 1.88488e+06, step = 16801 (0.207 sec)
INFO:tensorflow:global_step/sec: 469.102
INFO:tensorflow:loss = 823697.0, step = 16901 (0.213 sec)
INFO:tensorflow:global_step/sec: 368.488
INFO:tensorflow:loss = 1.53954e+06, step = 17001 (0.273 sec)
INFO:tensorflow:global_s

DNNRegressor(params={'hidden_units': [50, 30, 10], 'embedding_lr_multipliers': None, 'dropout': None, 'feature_columns': (_RealValuedColumn(column_name='length', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='width', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='height', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='horsepower', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _OneHotColumn(sparse_id_column=_SparseColumnHashed(column_name='make', is_integerized=False, bucket_size=50, lookup_config=None, combiner='sum', dtype=tf.string)), _OneHotColumn(sparse_id_column=_SparseColumnKeys(column_name='num-of-doors', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('two', 'four'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.s

## Evaluating

In [117]:
dnnregressor.evaluate(input_fn=eval_input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-15-14:09:38
INFO:tensorflow:Restoring parameters from tensorboard/DNN_regressor/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2017-06-15-14:09:39
INFO:tensorflow:Saving dict for global step 20000: global_step = 20000, loss = 1.54576e+07


{'global_step': 20000, 'loss': 15457560.0}

## Predicting

In [118]:
preds = list(dnnregressor.predict(input_fn=eval_input_fn))

for i in range(TEST_DATA_SIZE):
    print('prediction:', preds[i], 'real value:', test_label.iloc[i])

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from tensorboard/DNN_regressor/model.ckpt-20000
prediction: 8559.48 real value: 10698.0
prediction: 10046.9 real value: 9988.0
prediction: 9907.54 real value: 10898.0
prediction: 10046.9 real value: 11248.0
prediction: 24726.5 real value: 16558.0
prediction: 24726.5 real value: 15998.0
prediction: 20584.9 real value: 15690.0
prediction: 20584.9 real value: 15750.0
prediction: 3707.19 real value: 7775.0
prediction: 5748.44 real value: 7975.0


### Creating an Experiment

In [82]:
# @1.2.0 experiment_fn(run_config, params) - > experiment_fn(output_dir)
def experiment_fn(output_dir):
    # This function makes an Experiment, containing an Estimator and inputs for training and evaluation.
    # You can use params and config here to customize the Estimator depending on the cluster or to use
    # hyperparameter tuning.

    # Collect information for training
    # @1.2.0 config=run_config -> ''
    return tf.contrib.learn.Experiment(estimator=tf.contrib.learn.LinearRegressor(
                                     feature_columns=linear_features, model_dir=output_dir),
                                     train_input_fn=training_input_fn,
                                     train_steps=10000,
                                     eval_input_fn=eval_input_fn)

In [83]:
import shutil
# @1.2.0 tf.contrib.learn.learn_runner(exp, run_config=tf.contrib.learn.RunConfig(model_dir="/tmp/output_dir")
# -> tf.contrib.learn.python.learn.learm_runner.run(exp, output_dir='/tmp/output_dir')
shutil.rmtree("/tmp/output_dir", ignore_errors=True)

from tensorflow.contrib.learn.python.learn import learn_runner
learn_runner.run(experiment_fn, output_dir='/tmp/output_dir')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_session_config': None, '_model_dir': '/tmp/output_dir', '_master': '', '_tf_random_seed': None, '_environment': 'local', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faf2c6dc5f8>, '_save_summary_steps': 100, '_evaluation_master': '', '_is_chief': True}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor 

INFO:tensorflow:loss = 2.04795e+08, step = 3301 (0.208 sec)
INFO:tensorflow:global_step/sec: 485.621
INFO:tensorflow:loss = 2.34377e+08, step = 3401 (0.206 sec)
INFO:tensorflow:global_step/sec: 486.705
INFO:tensorflow:loss = 2.8409e+08, step = 3501 (0.205 sec)
INFO:tensorflow:global_step/sec: 487.064
INFO:tensorflow:loss = 2.64078e+08, step = 3601 (0.205 sec)
INFO:tensorflow:global_step/sec: 479.578
INFO:tensorflow:loss = 1.93784e+08, step = 3701 (0.209 sec)
INFO:tensorflow:global_step/sec: 477.774
INFO:tensorflow:loss = 2.8074e+08, step = 3801 (0.209 sec)
INFO:tensorflow:global_step/sec: 485.669
INFO:tensorflow:loss = 2.33637e+08, step = 3901 (0.206 sec)
INFO:tensorflow:global_step/sec: 486.921
INFO:tensorflow:loss = 1.72349e+08, step = 4001 (0.205 sec)
INFO:tensorflow:global_step/sec: 485.449
INFO:tensorflow:loss = 2.2439e+08, step = 4101 (0.206 sec)
INFO:tensorflow:global_step/sec: 485.478
INFO:tensorflow:loss = 3.11015e+08, step = 4201 (0.206 sec)
INFO:tensorflow:global_step/sec: 4

INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]

({'global_step': 10000, 'loss': 2.0313659e+08}, [])